In [1]:
import yaml
import copy
import os

In [4]:
analysis_config={'output_dir': 'data/experiments/pt1_ft2-1/analysis_higher/world_distance_l5',
 'experiment_dir': 'data/experiments/pt1_ft2-1',
 'cities_csv': 'data/datasets/cities/cities.csv',
 'layers': [5],
 'n_train_cities': 3250,
 'n_test_cities': 1250,
 'prompt_format': 'dist_firstcity_last_and_comma',
 'method': {'name': 'linear'},
 'probe_train': 'region:.* && city_id:^[1-9][0-9]{3,}$',
 'probe_test': 'region:.* && city_id:^[1-9][0-9]{3,}$',
 'device': 'cuda',
 'seed': 42,
 'save_repr_ckpts': [-2],
 'perform_pca': True}

In [ ]:
all_yaml_paths={}
for exp_name in ["pt1",*["pt1_ft1-"+str(i) for i in [1,2,3,4,5,6,7]],*["pt1_ft2-"+str(i) for i in [1,2,3,4,5,6,7]],"pt1_ft3-1"]
        fol = f"./{exp_name}"
        os.makedirs(fol, exist_ok=True)
        print(exp_name)
        
        yaml_paths=[]
        config = copy.deepcopy(analysis_config)
        config["output_dir"] = f'/n/home12/cfpark00/WM_1/data/experiments/{exp_name}/analysis_higher/world_distance_l5'
        config["experiment_dir"] = f'data/experiments/{exp_name}'
        yaml_path=os.path.join(fol, "multi_task.yaml")
        with open(yaml_path, "w") as f:
            yaml.dump(config, f)
        yaml_paths.append(yaml_path)
        
        for task in ["distance","trianglearea","angle","compass","inside","perimeter","crossing"]:
            for atlantis in [False,True]:
                taskname=("atlantis_"+task) if atlantis else task
                config=copy.deepcopy(task_config)
                config["output_dir"] = f'/n/home12/cfpark00/WM_1/data/experiments/{exp_name}/evals/{taskname}'
                config["experiment_dir"] = f'data/experiments/{exp_name}'
                if atlantis:
                    config["dataset_path"]=f'data/datasets/{task}_100k_atlantis_required'
                else:
                    config["dataset_path"]=f'data/datasets/{task}_1M_no_atlantis'
                yaml_path=os.path.join(fol, f"{taskname}.yaml")
                with open(yaml_path, "w") as f:
                    yaml.dump(config, f)
                yaml_paths.append(yaml_path)
        
        bashfilebase="/n/home12/cfpark00/WM_1/scripts/eval/ftset"
        bashfilepath=os.path.join(bashfilebase,"eval_"+exp_name+".sh")
        with open(bashfilepath,"w") as f:
            for yaml_path in yaml_paths:
                f.write(f"uv run python src/eval/evaluate_checkpoints.py {os.path.abspath(yaml_path)}\n")